In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ROOT_DIR = '.'
DATA_DIR = os.path.join(ROOT_DIR, 'data')
PRES_DIR = os.path.join(DATA_DIR, '')
os.chdir(ROOT_DIR)


COUNTYTAX_FNAME = '18incyallagi.csv'
PRES_FNAME = 'presidential.csv'

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

import warnings
warnings.filterwarnings('ignore')


The FIPS (Federal Information Processing Standard) code is used to identify counties. It seems that the national FIPS code is constructed from the state and county FIPS codes. It is not clear to what extent FIPS codes are invariant historically but at least for current tax and electorial data, we can trust them.

In [3]:
def make_national_fips(state_fips, county_fips):

    state_fips = str(state_fips)
    county_fips = str(county_fips)

    n = len(county_fips)
    county_fips = '0'*(3-n) + county_fips
    
    national_fips = state_fips + county_fips
    
    return national_fips

# Presidential Election Results

In [4]:
os.listdir(PRES_DIR)
path = os.path.join(PRES_DIR, PRES_FNAME)

df_pres = pd.read_csv(path)

In [5]:
df_pres[:3]

,fips,name,votes,absentee_votes,reporting,precincts,absentee_method,eevp,tot_exp_vote,eevp_value,eevp_display,eevp_source,turnout_stage,absentee_count_progress,absentee_outstanding,absentee_max_ballots,provisional_outstanding,provisional_count_progress,results_trumpd,results_bidenj,results_jorgensenj,results_venturaj,results_blankenshipd,results_pierceb,results_de_la_fuenter,results_absentee_trumpd,results_absentee_bidenj,results_absentee_jorgensenj,results_absentee_venturaj,results_absentee_blankenshipd,results_absentee_pierceb,results_absentee_de_la_fuenter,last_updated,leader_margin_value,leader_margin_display,leader_margin_name_display,leader_party_id,margin2020,votes2016,margin2016,state,retrieved_time,results_hawkinsh,results_write_ins,results_absentee_hawkinsh,results_absentee_write_ins,votes2012,margin2012,results_westk,results_kennedya,...,results_simmonsj,results_boddiep,results_hammonsb,results_hoeflingt,results_absentee_simmonsj,results_absentee_boddiep,results_absentee_hammonsb,results_absentee_hoeflingt,results_collinsp,results_absentee_collinsp,results_kingr,results_absentee_kingr,results_segalj,results_absentee_segalj,results_gammonc,results_myersj,results_absentee_gammonc,results_absentee_myersj,results_paigeh,results_lafontainec,results_duncanr,results_mccormick,results_huberb,results_kopitkek,results_swingg,results_scalfz,results_absentee_paigeh,results_absentee_lafontainec,results_absentee_duncanr,results_absentee_mccormick,results_absentee_huberb,results_absentee_kopitkek,results_absentee_swingg,results_absentee_scalfz,results_hunterd,results_absentee_hunterd,results_none_of_these_candidates,results_absentee_none_of_these_candidates,results_tittles,results_absentee_tittles,results_charlesm,results_mchughj,results_jacob_fambrop,results_scottj,results_kishorej,results_absentee_charlesm,results_absentee_mchughj,results_absentee_jacob_fambrop,results_absentee_scottj,results_absentee_kishorej
0,2901,ED 1,7064,3420,8,9,Released as a separate subtotal as first repor...,93.0,7634.0,93%,93% reported,edison,2.0,unknown,NaN,4000.0,NaN,NaN,3387,3363,207,49.0,32.0,24.0,2.0,1290,2024,59,26.0,16.0,5.0,0.0,2020-11-11T13:18:16Z,0.33975,R+0.34,Trump +0.34,republican,0.33975,6638.0,9.1,alaska,2020-11-13T00:38:32.233223Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2910,ED 10,9911,4778,10,11,Released as a separate subtotal as first repor...,95.0,10396.0,95%,95% reported,edison,2.0,unknown,NaN,4000.0,NaN,NaN,7106,2462,252,53.0,20.0,15.0,3.0,2941,1707,99,17.0,9.0,5.0,0.0,2020-11-11T03:25:58Z,46.85703,R+47,Trump +47,republican,46.85703,9040.0,49.2,alaska,2020-11-13T00:38:32.233223Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2911,ED 11,9740,5425,7,9,Released as a separate subtotal as first repor...,87.0,11142.0,87%,87% reported,edison,2.0,unknown,NaN,4000.0,NaN,NaN,6497,2897,232,74.0,21.0,14.0,5.0,3163,2080,119,42.0,12.0,6.0,3.0,2020-11-11T13:18:21Z,36.96099,R+37,Trump +37,republican,36.96099,9689.0,44.4,alaska,2020-11-13T00:38:32.233223Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
len(df_pres['fips'].unique())

3159

In [7]:
# Lets look at California for example
df_pres[df_pres['state']=='california']['fips'].unique()

array([6001, 6003, 6005, 6007, 6009, 6011, 6013, 6015, 6017, 6019, 6021,
       6023, 6025, 6027, 6029, 6031, 6033, 6035, 6037, 6039, 6041, 6043,
       6045, 6047, 6049, 6051, 6053, 6055, 6057, 6059, 6061, 6063, 6065,
       6067, 6069, 6071, 6073, 6075, 6077, 6079, 6081, 6083, 6085, 6087,
       6089, 6091, 6093, 6095, 6097, 6099, 6101, 6103, 6105, 6107, 6109,
       6111, 6113, 6115])

In [8]:
# There are many columns from the presidential election data
# let's keep just a few for now. For example, we keep only
# Trump-Biden, ignoring third-party candidates.

cols = ['fips', 'votes', 'absentee_votes', 
        'results_trumpd', 'results_bidenj', 
        'results_absentee_trumpd', 
        'results_absentee_bidenj', 'leader_margin_value', 
        'leader_party_id']

df_pres_clean = df_pres[cols]

In [9]:
# How many votes each candidate got in each state
df_pres_states = df_pres.groupby('state')[['results_trumpd', 'results_bidenj']].sum().reset_index()

In [10]:
df_pres_states[:5]

,state,results_trumpd,results_bidenj
0,alabama,1434159,843473
1,alaska,160193,115543
2,arizona,1655461,1666495
3,arkansas,757078,418069
4,california,5491372,10401707


# Tax

ammounts appear to be in US$1000's  

Key columns are  

Number of returns -- 'N1'  

Total Income:  
number -- 'N02650'  
ammount -- 'A02650'  

Total Tax Liability:  
number -- 'N10300'  
ammount -- 'A10300'  

In [11]:
path = os.path.join(DATA_DIR, COUNTYTAX_FNAME)
df_tax = pd.read_csv(path)

# There are many columns, at least one for every line on the 1040 form!
df_tax[:2]

,STATEFIPS,STATE,COUNTYFIPS,COUNTYNAME,agi_stub,N1,mars1,MARS2,MARS4,ELF,CPREP,PREP,DIR_DEP,N2,NUMDEP,TOTAL_VITA,VITA,TCE,VITA_EIC,RAC,ELDERLY,A00100,N02650,A02650,N00200,A00200,N00300,A00300,N00600,A00600,N00650,A00650,N00700,A00700,N00900,A00900,N01000,A01000,N01750,A01750,SCHF,N02300,A02300,N02500,A02500,N26270,A26270,N02900,A02900,N03220,...,N07100,A07100,N07300,A07300,N07180,A07180,N07230,A07230,N07240,A07240,N07225,A07225,N07260,A07260,N09400,A09400,N85770,A85770,N85775,A85775,N09750,A09750,N10600,A10600,N59660,A59660,N59720,A59720,N11070,A11070,N10960,A10960,N11560,A11560,N06500,A06500,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11900,A11900,N11902,A11902,N12000,A12000
0,1,AL,0,Alabama,1,25180.0,13340.0,9780.0,670.0,18510.0,3260.0,16860.0,6620.0,39820.0,5740.0,150.0,110.0,40.0,0.0,540.0,13180.0,-1595100.0,18010.0,-1575833.0,4770.0,189188.0,7840.0,29870.0,4110.0,27437.0,3750.0,18905.0,550.0,1553.0,9610.0,-167997.0,5930.0,73482.0,4510.0,73162.0,3260.0,0.0,0.0,0.0,0.0,4310.0,-377064.0,4070.0,19267.0,40.0,...,50.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,9.0,0.0,0.0,2440.0,6253.0,4050.0,46255.0,4350.0,47699.0,0.0,0.0,11790.0,54862.0,1320.0,1630.0,550.0,744.0,420.0,879.0,780.0,733.0,2060.0,2048.0,610.0,550.0,3900.0,8997.0,90.0,81.0,0.0,0.0,2230.0,3311.0,10810.0,48261.0,10210.0,38603.0,720.0,7308.0
1,1,AL,0,Alabama,2,257140.0,202060.0,22490.0,29670.0,226630.0,16340.0,122140.0,156640.0,253910.0,62100.0,9140.0,5960.0,3180.0,2430.0,37420.0,46740.0,1396297.0,257140.0,1426504.0,198080.0,1138465.0,32100.0,17484.0,15600.0,20913.0,14220.0,14164.0,1800.0,1334.0,39500.0,116475.0,13280.0,11185.0,29950.0,168380.0,2040.0,2490.0,6398.0,410.0,1714.0,2680.0,-13531.0,35790.0,30207.0,230.0,...,1150.0,262.0,440.0,16.0,0.0,0.0,60.0,18.0,0.0,0.0,220.0,87.0,0.0,0.0,32000.0,24958.0,12740.0,125572.0,13480.0,129808.0,0.0,0.0,221000.0,223866.0,87830.0,112341.0,78270.0,99863.0,29500.0,21723.0,11970.0,10581.0,5620.0,3676.0,6080.0,1899.0,39790.0,27588.0,0.0,0.0,0.0,0.0,21540.0,10722.0,209960.0,207096.0,209020.0,204442.0,1180.0,2703.0


In [12]:
# Lets look at california.
# This appears to be about $510 billion in total tax paid
# This looks like about twice what I find quoted online
df_tax[df_tax['STATE']=='CA']['A10300'].sum()

510230672.0

In [13]:
# Ah the reason is that the tax data includes the whole state as a county, lets remove it
df_tax_states = df_tax[df_tax['COUNTYFIPS']==0]
df_tax_states = df_tax_states.groupby('STATE')['A10300'].sum().reset_index()
df_tax_states[:5]

,STATE,A10300
0,AK,3173902.0
1,AL,14112810.0
2,AR,8657796.0
3,AZ,26427580.0
4,CA,255115336.0


In [14]:
# Now california is around $255 billion, which is closer to what I found online.

In [15]:
# I get the total federal income tax to be $1,622 billion
# On wikipedia 
# https://en.wikipedia.org/wiki/United_States_federal_budget#Major_receipt_categories
# I found $1,684B quoted, which is close
df_tax_states['A10300'].sum()

1622166007.0

In [19]:
# lets add the national fips to the tax data

df_tax['NATIONAL_FIPS'] = df_tax.apply(lambda row: 
                        make_national_fips(row['STATEFIPS'], row['COUNTYFIPS']), axis=1)

# again, exclude entire states, just leep legitimate counties
df_tax_cty = df_tax[df_tax['COUNTYFIPS']!=0]

states = df_tax['STATE'].unique()
dfs_state = {state: df_tax[df_tax['STATE']==state] for state in states}

# Join Election and Tax data

A fly in the ointment is that Alaska and DC are hard to nail down.  

Im going to remove them as their contribution should be smallish.

In [ ]:
bad_states = ['AK', 'alaska', 'Alaska', 'District of Columbia', 'DC', 'district-of-columbia']

pres_clean_cols = ['fips', 'state', 'results_trumpd', 'results_bidenj']
df_pres_clean = df_pres[~df_pres['state'].isin(bad_states)][pres_clean_cols]
df_tax_clean = df_tax_cty[~df_tax_cty['STATE'].isin(bad_states)]

We also need to align the 'fips' columns between the two data sets so we can merge

In [39]:
df_pres_clean['fips'] = df_pres_clean['fips'].astype(int)
df_tax_clean = df_tax_clean.rename({'NATIONAL_FIPS':'fips'}, axis=1)
df_tax_clean['fips'] = df_tax_clean['fips'].astype(int)

In [40]:
pres_fips = df_pres_clean['fips'].unique()
tax_fips = df_tax_clean['fips'].unique()

In [61]:
# Here we just check that our fips codes are consistent

missing_fips_1 = [fips for fips in tax_fips if fips not in pres_fips]
missing_fips_2 = [fips for fips in pres_fips if fips not in tax_fips]
print('missing_fips_1', missing_fips_1)
print('missing_fips_2', missing_fips_2)

missing_fips_1 []
missing_fips_2 []


In [45]:
# This is the total tax paid by each county
rename_dict = {'A10300':'total_tax_liability_county'}
df_total_tax_county = df_tax_clean.groupby(['fips'])['A10300'].sum().reset_index().rename(rename_dict, axis=1)
df_total_tax_county[:2]

,fips,total_tax_liability_county
0,1001,131036.0
1,1003,858857.0


Now we can merge the tax and voting data  
So we have for each county, the tax paid and the number of votes for Biden/Trump.

In [49]:
df_data = df_total_tax_county.merge(df_pres_clean, on=['fips'], how='left')
df_data[:2]

,fips,total_tax_liability_county,state,results_trumpd,results_bidenj
0,1001,131036.0,alabama,19764,7450
1,1003,858857.0,alabama,83055,24344


This is where we make some assumptions.

We assume that in every county, each voter has paid the same ammount of tax,
obviously this is not strictly true but statistically, it should be far more 
true at the county level than at the national level. Indeed this is the the essential point of the current analysis.

So using this assumption, we say that the tax attributed to Trump/Biden is the total
tax paid by that county multiplied by the percentage of votes which Trump/Biden received.

In [ ]:
df_data['Trump_pc'] = df_data['results_trumpd']/(df_data['results_trumpd']+ df_data['results_bidenj'])
df_data['Biden_pc'] = df_data['results_bidenj']/(df_data['results_trumpd']+ df_data['results_bidenj'])
df_data['Trump_tax'] = df_data['Trump_pc'] * df_data['total_tax_liability_county']
df_data['Biden_tax'] = df_data['Biden_pc'] * df_data['total_tax_liability_county']

In [55]:
# We now um over all counties to get the total federal tax attributed to each candidate

Trump_tax_cty = df_data['Trump_tax'].sum()
Biden_tax_cty = df_data['Biden_tax'].sum()
print('Trump_tax: $', Trump_tax_cty/10**6, 'billion')
print('Biden_tax: $', Biden_tax_cty/10**6, 'billion')

Trump_tax: $ 678.1870617226891 billion
Biden_tax: $ 934.0734622773111 billion


In [62]:
# So Biden voters paid much more tax but there is also more of them! 
# So lets look at how much tax each voter paid

num_trump_voters = df_data['results_trumpd'].sum()
num_biden_voters = df_data['results_bidenj'].sum()
print('num_trump_voters: ', num_trump_voters)
print('num_biden_voters: ', num_biden_voters)

trump_tax_per_cp_cty = Trump_tax_cty/num_trump_voters
biden_tax_per_cp_cty = Biden_tax_cty/num_biden_voters
print('trump_tax_per_voter: $', trump_tax_per_cp_cty *10**3, )
print('biden_tax_per_voter: $', biden_tax_per_cp_cty *10**3)

num_trump_voters:  72366111
num_biden_voters:  77393834
trump_tax_per_voter: $ 9371.611274270206
biden_tax_per_voter: $ 12069.094060869385
